In [2]:
import random
import numpy as np
from tensorflow import random as tf_rnd

In [3]:
np.random.seed(9)
random.seed(9)
tf_rnd.set_seed(9)

In [4]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [6]:
from tensorflow import keras

In [7]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

In [8]:
df_hongos = pd.read_csv("/content/drive/MyDrive/datosTP2/files/MushroomDataset/secondary_data.csv", delimiter=";")

Me quedo con 80% de los datos para train, 10% para validation y 10% para test. \\
Los separo al azar para obtener datos que representen bien todo el conjunto.

In [9]:
y = df_hongos["class"]
X = df_hongos.drop("class", axis = 1)
tamanio_test = 0.1

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=tamanio_test, random_state=9)

In [10]:
tamanio_validation = 0.1 / (1 - tamanio_test)

In [11]:
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=tamanio_validation, random_state=9)

##**Imputación de nulos**

In [12]:
df_hongos.nunique()

class                      2
cap-diameter            2571
cap-shape                  7
cap-surface               11
cap-color                 12
does-bruise-or-bleed       2
gill-attachment            7
gill-spacing               3
gill-color                12
stem-height             2226
stem-width              4630
stem-root                  5
stem-surface               8
stem-color                13
veil-type                  1
veil-color                 6
has-ring                   2
ring-type                  8
spore-print-color          7
habitat                    8
season                     4
dtype: int64

Veo cuantos nulos hay por columna

In [13]:
df_hongos.count()

class                   61069
cap-diameter            61069
cap-shape               61069
cap-surface             46949
cap-color               61069
does-bruise-or-bleed    61069
gill-attachment         51185
gill-spacing            36006
gill-color              61069
stem-height             61069
stem-width              61069
stem-root                9531
stem-surface            22945
stem-color              61069
veil-type                3177
veil-color               7413
has-ring                61069
ring-type               58598
spore-print-color        6354
habitat                 61069
season                  61069
dtype: int64

Según la descripción de los datos, sé que el valor "f" puede usarse tanto para nombrar un dato válido, como para definir un "None". En las columnas donde no sea un dato válido, los voy a reemplazar por "*" que es mi convención para los valores desconocidos.

In [14]:
columnas_con_variable_f = df_hongos.select_dtypes(include=["object"]).columns.map(lambda x: x if df_hongos[x].str.contains("f").any() else None).dropna() 
columnas_con_variable_f

Index(['cap-shape', 'does-bruise-or-bleed', 'gill-attachment', 'gill-spacing',
       'gill-color', 'stem-root', 'stem-surface', 'stem-color', 'has-ring',
       'ring-type'],
      dtype='object')

###**Nulos en columnas de gill-":**

In [15]:
columnas_gill =  ["gill-attachment", "gill-spacing", "gill-color"]

In [16]:
df_hongos.loc[ : , columnas_gill ]   

,gill-attachment,gill-spacing,gill-color
0,e,NaN,w
1,e,NaN,w
2,e,NaN,w
3,e,NaN,w
4,e,NaN,w
...,...,...,...
61064,f,f,f
61065,f,f,f
61066,f,f,f
61067,f,f,f


In [17]:
df_hongos.loc[ : , columnas_gill ].loc[ df_hongos["gill-attachment"] == "f"] 

,gill-attachment,gill-spacing,gill-color
54362,f,f,f
54363,f,f,f
54364,f,f,f
54365,f,f,f
54366,f,f,f
...,...,...,...
61064,f,f,f
61065,f,f,f
61066,f,f,f
61067,f,f,f


In [18]:
df_hongos.loc[ : , columnas_gill ].loc[ df_hongos["gill-attachment"] == "f"].nunique() 

gill-attachment    1
gill-spacing       1
gill-color         1
dtype: int64

In [19]:
df_hongos.loc[ : , columnas_gill ].loc[ df_hongos["gill-spacing"] == "f"].nunique() 

gill-attachment    1
gill-spacing       1
gill-color         1
dtype: int64

In [20]:
df_hongos.loc[ : , columnas_gill ].loc[ df_hongos["gill-color"] == "f"].nunique() 

gill-attachment    1
gill-spacing       1
gill-color         1
dtype: int64

En lo anterior puedo ver que las columnas con gill son siempre "f" todas a la vez. Supongo así que el gill no es desconocido, sino que no lo tiene el hongo. \\
Entonces dejo esa variable "f" tal como está en esas columnas porque me da información.

###**Nulos en columnas de stem- :**

In [21]:
df_hongos["stem-height"].sort_values(ascending=True)

57818     0.00
56566     0.00
56565     0.00
56564     0.00
56563     0.00
         ...  
2881     32.85
2897     33.03
2937     33.25
3143     33.72
2870     33.92
Name: stem-height, Length: 61069, dtype: float64

In [22]:
df_hongos["stem-width"].sort_values(ascending=True)

57948      0.00
57845      0.00
57844      0.00
57843      0.00
57842      0.00
          ...  
48501     95.68
48447     98.00
48366    101.69
48682    102.48
48597    103.91
Name: stem-width, Length: 61069, dtype: float64

In [23]:
df_hongos.loc[ : , ["stem-root", "stem-surface", "stem-color"] ].loc[ df_hongos["stem-height"] == 0.00] 

,stem-root,stem-surface,stem-color
56480,f,f,f
56481,f,f,f
56482,f,f,f
56483,f,f,f
56484,f,f,f
...,...,...,...
58240,f,f,f
58241,f,f,f
58242,f,f,f
58243,f,f,f


In [24]:
df_hongos.loc[ : , ["stem-root", "stem-surface", "stem-color"] ].loc[ df_hongos["stem-width"] == 0.00] 

,stem-root,stem-surface,stem-color
56480,f,f,f
56481,f,f,f
56482,f,f,f
56483,f,f,f
56484,f,f,f
...,...,...,...
58240,f,f,f
58241,f,f,f
58242,f,f,f
58243,f,f,f


In [25]:
df_hongos.loc[ : , ["stem-root", "stem-surface", "stem-color"] ].loc[ (df_hongos["stem-height"] == 0.00) & (df_hongos["stem-width"] == 0.00)]

,stem-root,stem-surface,stem-color
56480,f,f,f
56481,f,f,f
56482,f,f,f
56483,f,f,f
56484,f,f,f
...,...,...,...
58240,f,f,f
58241,f,f,f
58242,f,f,f
58243,f,f,f


In [26]:
df_hongos.loc[ : , ["stem-root", "stem-surface", "stem-color"] ].loc[ (df_hongos["stem-height"] == 0.00) & (df_hongos["stem-width"] == 0.00)].nunique()

stem-root       1
stem-surface    1
stem-color      1
dtype: int64

In [27]:
df_hongos.loc[(df_hongos["stem-height"] == 0.00) & (df_hongos["stem-width"] == 0.00)].head(3)

,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
56480,p,2.48,o,t,n,f,NaN,c,w,0.0,...,f,f,f,NaN,NaN,f,f,n,d,u
56481,p,4.29,o,t,w,f,NaN,c,w,0.0,...,f,f,f,NaN,NaN,f,f,n,d,u
56482,p,4.29,o,t,n,f,NaN,c,w,0.0,...,f,f,f,NaN,NaN,f,f,n,d,u


Veo que cuando no tienen valor los tallos de los hongos, el valor es siempre "f", que lo supongo desconocido. \\
En el paper decía que secondary_data solo tenía hongos con tallo, por lo que voy a remplazar los valores de tallos inválidos por la media de la columna en todo el dataset, sabiendo que solo valen 0.00 los valores de altura y ancho del tallo para ambos a la vez.

In [28]:
promedio_alto_tallo_dataset = df_hongos["stem-height"].mean()
promedio_ancho_tallo_dataset = df_hongos["stem-width"].mean()


X_train["stem-height"] = X_train["stem-height"].replace(0.00, promedio_alto_tallo_dataset)
X_validation["stem-height"] = X_validation["stem-height"].replace(0.00, promedio_alto_tallo_dataset)
X_test["stem-height"] = X_test["stem-height"].replace(0.00, promedio_alto_tallo_dataset)

X_train["stem-width"] = X_train["stem-width"].replace(0.00, promedio_ancho_tallo_dataset)
X_validation["stem-width"] = X_validation["stem-width"].replace(0.00, promedio_ancho_tallo_dataset)
X_test["stem-width"] = X_test["stem-width"].replace(0.00, promedio_ancho_tallo_dataset)

###**Nulos en columna has-ring y ring-type:**

In [29]:
df_hongos["has-ring"].unique()

array(['t', 'f'], dtype=object)

In [30]:
df_hongos["ring-type"].loc[ df_hongos["has-ring"] == "f" ].unique()

array(['f'], dtype=object)

In [31]:
df_hongos["ring-type"].loc[ df_hongos["has-ring"] == "t" ].unique()

array(['g', 'p', 'e', 'l', 'm', nan, 'r', 'f', 'z'], dtype=object)

In [32]:
df_hongos["ring-type"].loc[ (df_hongos["has-ring"] == "t") & (df_hongos["ring-type"] == "f") ]

30005    f
30006    f
30007    f
30008    f
30009    f
        ..
59652    f
59653    f
59654    f
59655    f
59656    f
Name: ring-type, Length: 2471, dtype: object

Veo en lo anterior que el "f" de ring-type y has-ring lo puedo suponer como desconocido. Esto porque veo que no significa que no tenga anillo por ejemplo.

###**Nulos en columna veil-type:**

In [33]:
df_hongos["veil-type"].unique()

array(['u', nan], dtype=object)

Supongo que los NaN de "veil-type" son el otro valor posible ("p").

In [34]:
X_train["veil-type"] = X_train["veil-type"].fillna("p")
X_validation["veil-type"] = X_validation["veil-type"].fillna("p")
X_test["veil-type"] = X_test["veil-type"].fillna("p")

###**Asignación de constante a valores nan y "f" desconocidos:**

Remplazo todos los NaN restantes por *. Esto porque en el texto y en los gráficos no encontré nada para suponer otra cosa. \\
Uso * porque supongo que no lo van a pasar nunca como dato de una columna.



In [35]:
X_train = X_train.fillna("*")
X_validation = X_validation.fillna("*")
X_test = X_test.fillna("*")

También reemplazo todos los "f" que supongo desconocidos por el mismo "*" para tener una convención más uniforme para los desconocidos o nulos. \\
No uso "f" porque otras columnas lo usan con un significado real.

In [36]:
columnas_con_f_desconocidos = ["stem-root", "stem-surface", "stem-color", "has-ring", "ring-type"]


X_train[columnas_con_f_desconocidos] = X_train[columnas_con_f_desconocidos].replace("f", "*")
X_validation[columnas_con_f_desconocidos] = X_validation[columnas_con_f_desconocidos].replace("f", "*")
X_test[columnas_con_f_desconocidos] = X_test[columnas_con_f_desconocidos].replace("f", "*")

##**Encoding y normalización de datos:**

In [37]:
columnas_categoricas = df_hongos.select_dtypes(include=["object"]).columns.drop("class")

In [38]:
one_hot_encoder = OneHotEncoder(sparse_output=False, drop="if_binary", dtype=int)

X_train_encoded = one_hot_encoder.fit_transform( X_train[columnas_categoricas] )
X_validation_encoded = one_hot_encoder.transform( X_validation[columnas_categoricas] )
X_test_encoded = one_hot_encoder.transform( X_test[columnas_categoricas] )

Normalizo los datos de las columnas numéricas.

In [39]:
scaler = StandardScaler()

nombres_features = one_hot_encoder.get_feature_names_out(columnas_categoricas)


df_columnas_numericas = X_train[["cap-diameter", "stem-height", "stem-width"]].reset_index(drop=True)
df_columnas_numericas_normalizadas = pd.DataFrame(scaler.fit_transform(df_columnas_numericas))

X_train_encoded = pd.DataFrame(X_train_encoded, columns=nombres_features)
X_train_encoded = pd.concat([X_train_encoded, df_columnas_numericas_normalizadas], axis=1).values



df_columnas_numericas = X_validation[["cap-diameter", "stem-height", "stem-width"]].reset_index(drop=True)
df_columnas_numericas_normalizadas = pd.DataFrame(scaler.transform(df_columnas_numericas))

X_validation_encoded = pd.DataFrame(X_validation_encoded, columns=nombres_features)
X_validation_encoded = pd.concat([X_validation_encoded, df_columnas_numericas_normalizadas], axis=1).values



df_columnas_numericas = X_test[["cap-diameter", "stem-height", "stem-width"]].reset_index(drop=True)
df_columnas_numericas_normalizadas = pd.DataFrame(scaler.transform(df_columnas_numericas))

X_test_encoded = pd.DataFrame(X_test_encoded, columns=nombres_features)
X_test_encoded = pd.concat([X_test_encoded, df_columnas_numericas_normalizadas], axis=1).values

In [40]:
y_train_encoded = y_train.map(lambda x: 0 if x == 'e' else 1).values
y_validation_encoded = y_validation.map(lambda x: 0 if x == 'e' else 1).values
y_test_encoded = y_test.map(lambda x: 0 if x == 'e' else 1).values

##**Modelo y red neuronal:**

In [41]:
modelo = KNeighborsClassifier().fit(X_train_encoded, y_train_encoded)

In [42]:
input_shape = (X_train_encoded.shape[1],)

Uso una red con 3 capas, la última con activation="sigmoid" para que devuelva una probabilidad.

In [43]:
modelo = Sequential([
    Dense(4, activation="tanh", input_shape=input_shape),
    Dense(2, activation="tanh"),
    Dense(1, activation="sigmoid")

])
modelo.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 4)                 492       
                                                                 
 dense_1 (Dense)             (None, 2)                 10        
                                                                 
 dense_2 (Dense)             (None, 1)                 3         
                                                                 
Total params: 505
Trainable params: 505
Non-trainable params: 0
_________________________________________________________________


In [44]:
modelo.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate=0.001), metrics=["accuracy"])

In [45]:
modelo.fit(X_train_encoded, y_train_encoded)

1527/1527 [==============================] - 3s 2ms/step - loss: 0.4225 - accuracy: 0.8183


In [46]:
from sklearn.metrics import roc_auc_score

In [47]:
roc_auc_score(y_validation_encoded, modelo.predict(X_validation_encoded))

191/191 [==============================] - 0s 1ms/step


0.9607602248711125

In [48]:
roc_auc_score(y_test_encoded, modelo.predict(X_test_encoded))

191/191 [==============================] - 0s 1ms/step


0.9604064658828904